# Organism

In [1]:
! nvidia-smi

Wed Oct 14 13:34:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, ELU, LeakyReLU, Flatten, Dense, AveragePooling2D, Add
import pprint
pp = pprint.PrettyPrinter(indent=4)

import numpy as np
np.random.seed(666)
tf.random.set_seed(666)

In [3]:
options = {
    'a_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'a_include_BN': [True, False],
    'a_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'activation_type': [ReLU, ELU, LeakyReLU],
    'b_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'b_include_BN': [True, False],
    'b_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'include_pool': [True, False],
    'pool_type': [MaxPool2D, AveragePooling2D]
    # 'include_skip':[True, False]
    }

In [4]:
# Load the training and testing set of CIFAR10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train = X_train.astype('float32')
X_train = X_train/255.

X_test = X_test.astype('float32')
X_test = X_test/255.

y_train = tf.reshape(tf.one_hot(y_train, 10), shape=(-1, 10))
y_test = tf.reshape(tf.one_hot(y_test, 10), shape=(-1, 10))

# Create TensorFlow dataset
BATCH_SIZE = 1024
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(1024).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

170500096/170498071 [==============================] - 11s 0us/step


In [5]:
class Organism:

  def __init__(self, chromosome={}, phase=0):
    self.phase = phase
    self.chromosome = chromosome
    if len(self.chromosome) != 0:
      self.build_model()

  def build_model(self):
    if self.phase == 0:
      keras.backend.clear_session()
      inputs = Input(shape=(32,32,3))
      x = Conv2D(filters=self.chromosome['a_output_channels'],
                #  padding='same',
                 kernel_size=self.chromosome['a_filter_size'],
                 use_bias=self.chromosome['a_include_BN'])(inputs)
      if self.chromosome['a_include_BN']:
        x = BatchNormalization()(x)
      x = self.chromosome['activation_type']()(x)
      if self.chromosome['include_pool']:
        x = self.chromosome['pool_type']()(x)

      x = Conv2D(filters=self.chromosome['b_output_channels'],
                #  padding='same',
                 kernel_size=self.chromosome['b_filter_size'],
                 use_bias=self.chromosome['b_include_BN'])(x)
      if self.chromosome['b_include_BN']:
        x = BatchNormalization()(x)
      x = self.chromosome['activation_type']()(x)

      # if self.chromosome['include_skip']:
      #   stride = 2 if self.chromosome['include_pool'] else 1
      #   y = Conv2D(filters=self.chromosome['b_output_channels'],
      #              kernel_size=(1,1),
      #              strides=(stride, stride))(inputs)
      #   x = Add()([x,y])

      x = Flatten()(x)
      x = Dense(10, activation='softmax')(x)

      self.model = tf.keras.Model(inputs=[inputs], outputs=[x])
      self.model.compile(optimizer='adam',
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])
    else:
      print('Phase under construction')
      
  def fitnessFunction(self, train_ds, test_ds):
    self.model.fit(train_ds,
                   epochs=5,
                   verbose=1)
    _, self.fitness = self.model.evaluate(test_ds, verbose=0)
  
  def crossover(self, partner):
    child = Organism(chromosome={}, phase=0)
    endpoint = np.random.randint(low=0, high=len(self.chromosome))
    for idx, key in enumerate(self.chromosome):
      if idx <= endpoint:
        child.chromosome[key] = self.chromosome[key]
      else:
        child.chromosome[key] = partner.chromosome[key]
    child.build_model()
    return child
  
  def mutation(self):
    '''one of the gene is to be mutated'''
    index = np.random.randint(0, len(self.chromosome))
    key = list(self.chromosome.keys())[index]
    self.chromosome[key] = options[key][np.random.randint(len(options[key]))]
    self.build_model()
  
  def show(self):
    pp.pprint(self.chromosome)

# Generation

In [6]:
def random_hyper():
  return {
    'a_filter_size': options['a_filter_size'][np.random.randint(len(options['a_filter_size']))],
    'a_include_BN': options['a_include_BN'][np.random.randint(len(options['a_include_BN']))],
    'a_output_channels': options['a_output_channels'][np.random.randint(len(options['a_output_channels']))],
    'activation_type': options['activation_type'][np.random.randint(len(options['activation_type']))],
    'b_filter_size': options['b_filter_size'][np.random.randint(len(options['b_filter_size']))],
    'b_include_BN': options['b_include_BN'][np.random.randint(len(options['b_include_BN']))],
    'b_output_channels': options['b_output_channels'][np.random.randint(len(options['b_output_channels']))],
    'include_pool': options['include_pool'][np.random.randint(len(options['include_pool']))],
    'pool_type': options['pool_type'][np.random.randint(len(options['pool_type']))]
    # 'include_skip': options['include_skip'][np.random.randint(len(options['include_skip']))]
    }

def softmax(x):
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum()

In [7]:
class Generation:
	
	def __init__(self,
							fitSurvivalRate,
							unfitSurvivalProb,
							mutationRate,
							population_size):
		self.population_size = population_size
		self.population = []
		self.generation_number = 0
		self.mutationRate = mutationRate
		self.fitSurvivalRate = fitSurvivalRate
		self.unfitSurvivalProb = unfitSurvivalProb

		# creating the first population GENERATION 1
		# can be thought of as the setup function
		for idx in range(self.population_size):
			self.population.append(Organism(random_hyper(), phase=0))
		# calculates the fitness of all the organisms
		self.calcFitness(first=True)

	def calcFitness(self, first=False):
		# This function is used to calculate the fitness of all
		# the individuals of the population.
		if first:
			for individuals in self.population:
				individuals.fitnessFunction(train_ds, test_ds)
	 	# sort the population from high to low
		fitness = [ind.fitness for ind in self.population]
		sort_index = np.argsort(fitness)[::-1]
		self.population = [self.population[index] for index in sort_index]
	# genrate the children and place them in the previous generation
	# to make new generation
	def generate(self):
		number_of_fit = int(self.population_size * self.fitSurvivalRate)
		new_pop = self.population[:number_of_fit]
		for individual in self.population[number_of_fit:]:
			if np.random.rand() <= self.unfitSurvivalProb:
				new_pop.append(individual)
		for index, individual in enumerate(new_pop):
			if np.random.rand() <= self.mutationRate:
				new_pop[index].mutation()
		fitness = [ind.fitness for ind in new_pop]
		children=[]
		# create children with pairs
		for idx in range (self.population_size-len(new_pop)):
			parents = np.random.choice(new_pop, replace=False, size=(2,), p=softmax(fitness))
			A=parents[0]
			B=parents[1]
			child=A.crossover(B)
			child.fitnessFunction(train_ds, test_ds)
			children.append(child)
		self.generation_number+=1
		self.population = new_pop+children
	# evaluates the whole generation's fitness
	def evaluate(self):
		fitness = [ind.fitness for ind in self.population]
		print('Generation: {}'.format(self.generation_number))
		print('Best fitness: {:0.2f}'.format(fitness[0]))
		print('Average fitness: {:0.2f}'.format(sum(fitness)/len(fitness)))
		self.population[0].show()

In [8]:
population_size = 20
numer_generation = 5
fitSurvivalRate = 0.5
unfitSurvivalProb = 0.2
mutationRate = 0.1
generation = Generation(fitSurvivalRate=fitSurvivalRate,
                        unfitSurvivalProb=unfitSurvivalProb,
												mutationRate=mutationRate,
												population_size=population_size)

while generation.generation_number <= numer_generation:
	generation.generate()
	generation.calcFitness()
	generation.evaluate()

Epoch 1/5
49/49 [==============================] - 27s 546ms/step - loss: 4.6814 - accuracy: 0.2719
Epoch 2/5
49/49 [==============================] - 25s 500ms/step - loss: 1.8359 - accuracy: 0.4034
Epoch 3/5
49/49 [==============================] - 25s 506ms/step - loss: 1.7143 - accuracy: 0.4375
Epoch 4/5
49/49 [==============================] - 25s 509ms/step - loss: 1.5648 - accuracy: 0.4767
Epoch 5/5
49/49 [==============================] - 25s 508ms/step - loss: 1.5143 - accuracy: 0.4950
Epoch 1/5
49/49 [==============================] - 7s 145ms/step - loss: 1.8641 - accuracy: 0.3359
Epoch 2/5
49/49 [==============================] - 7s 140ms/step - loss: 1.5232 - accuracy: 0.4599
Epoch 3/5
49/49 [==============================] - 7s 141ms/step - loss: 1.4109 - accuracy: 0.5037
Epoch 4/5
49/49 [==============================] - 7s 141ms/step - loss: 1.3390 - accuracy: 0.5307
Epoch 5/5
49/49 [==============================] - 7s 140ms/step - loss: 1.2768 - accuracy: 0.5523
Epoch